# Lab 12-1 many to one
### word sentiment classification 
* many to one 
* variable input sequence length

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
from pprint import pprint
tf.set_random_seed(777)

### Prepairing dataset

In [2]:
words = ['good', 'bad', 'worse', 'so good']
y_data = [1,0,0,1]
char_set = ['<pad>'] + sorted(list(set(''.join(words))))
idx2char = {idx : char for idx, char in enumerate(char_set)}
char2idx = {char : idx for idx, char in enumerate(char_set)}

print(idx2char)
print(char2idx)

{0: '<pad>', 1: ' ', 2: 'a', 3: 'b', 4: 'd', 5: 'e', 6: 'g', 7: 'o', 8: 'r', 9: 's', 10: 'w'}
{'<pad>': 0, ' ': 1, 'a': 2, 'b': 3, 'd': 4, 'e': 5, 'g': 6, 'o': 7, 'r': 8, 's': 9, 'w': 10}


In [3]:
x_data = list(map(lambda word : [char2idx.get(char) for char in word], words))
x_data_len = list(map(lambda word : len(word), x_data))

print(x_data)
print(x_data_len)

[[6, 7, 7, 4], [3, 2, 4], [10, 7, 8, 9, 5], [9, 7, 1, 6, 7, 7, 4]]
[4, 3, 5, 7]


In [4]:
# padding
max_sequence = 10
x_data = pad_sequences(sequences = x_data, maxlen = max_sequence,
                       padding = 'post', truncating = 'post')

# 데이터 형태 확인
print(x_data)
print(x_data_len)
print(y_data)

[[ 6  7  7  4  0  0  0  0  0  0]
 [ 3  2  4  0  0  0  0  0  0  0]
 [10  7  8  9  5  0  0  0  0  0]
 [ 9  7  1  6  7  7  4  0  0  0]]
[4, 3, 5, 7]
[1, 0, 0, 1]


### Creating graph

In [5]:
# input
data = tf.data.Dataset.from_tensor_slices((x_data_len, x_data, y_data))
data = data.shuffle(buffer_size = 4)
data = data.batch(batch_size = 2)
data_iter = data.make_initializable_iterator()
x_mb_len, x_mb, y_mb = data_iter.get_next()
print(x_mb_len, x_mb, y_mb)

Tensor("IteratorGetNext:0", shape=(?,), dtype=int32) Tensor("IteratorGetNext:1", shape=(?, 10), dtype=int32) Tensor("IteratorGetNext:2", shape=(?,), dtype=int32)


In [6]:
# hyper-parameters for lstm (many to one), one-hot encoding
n_of_classes = 2
hidden_size = 10

x_mb_batch = keras.layers.Embedding(input_dim = len(idx2char), output_dim = len(idx2char),
                                    input_length = max_sequence,
                                    embeddings_initializer = keras.initializers.Identity,
                                    mask_zero = True, trainable = False)(x_mb)
entire_h, cell, final_h = keras.layers.LSTM(units = hidden_size, return_sequences = True,
                                            return_state = True)(x_mb_batch)
score = keras.layers.Dense(units = n_of_classes)(final_h)
loss = tf.losses.sparse_softmax_cross_entropy(labels = y_mb, logits = score)
prediction = tf.argmax(input = score, axis = -1)

In [7]:
# training
lr = .05
opt = tf.train.AdamOptimizer(learning_rate = lr)
training_op = opt.minimize(loss = loss)

### Training

In [8]:
sess_config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
sess = tf.Session(config = sess_config)
sess.run(tf.global_variables_initializer())

In [9]:
epochs = 10
tr_loss_hist = []

for epoch in range(epochs):
    
    sess.run(data_iter.initializer)
    avg_tr_loss = 0
    step = 0
    
    try:
        while True:
            _, tr_loss = sess.run([training_op, loss])
            avg_tr_loss += tr_loss
            step += 1
    except:
        avg_tr_loss /= step
        tr_loss_hist.append(avg_tr_loss)
        
    print('epoch : {:3}, loss : {:.3f}'.format(epoch + 1, avg_tr_loss))

epoch :   1, loss : 0.694
epoch :   2, loss : 0.431
epoch :   3, loss : 0.204
epoch :   4, loss : 0.045
epoch :   5, loss : 0.005
epoch :   6, loss : 0.001
epoch :   7, loss : 0.000
epoch :   8, loss : 0.000
epoch :   9, loss : 0.012
epoch :  10, loss : 0.000


### Accuracy

In [10]:
yhat = sess.run(prediction, feed_dict = {x_mb : x_data, x_mb_len : x_data_len})

In [11]:
print('accuracy : {:.2%}'.format(np.mean(yhat == y_data)))

accuracy : 100.00%
